In this cell, I imported the necessary libraries: **json, pandas, requests, and matplotlib.pyplot**. These will help with data handling, making API requests, and plotting visualizations.

In [1]:
import json
import pandas as pd
import requests
import matplotlib.pyplot as plt

Here, I made an API request to Open Charge Map to get electric vehicle (EV) charging stations in Australia. I used the provided API key in the request headers and checked if the response was successful by looking at the status code. If successful, I printed the first two entries of the JSON data to see the structure.

In [2]:
# API Request to Open Charge Map
url = "https://api.openchargemap.io/v3/poi/?output=json&countrycode=AU"

headers = {
    "X-API-Key": "2f5df05b-4b9d-46a3-b38e-c138e919abea"  # Replace 'YOUR_API_KEY' with your actual API key
}
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    print(data[:2])
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

[{'DataProvider': {'WebsiteURL': 'http://openchargemap.org', 'Comments': None, 'DataProviderStatusType': {'IsProviderEnabled': True, 'ID': 1, 'Title': 'Manual Data Entry'}, 'IsRestrictedEdit': False, 'IsOpenDataLicensed': True, 'IsApprovedImport': True, 'License': 'Licensed under Creative Commons Attribution 4.0 International (CC BY 4.0)', 'DateLastImported': None, 'ID': 1, 'Title': 'Open Charge Map Contributors'}, 'OperatorInfo': {'WebsiteURL': 'https://chargefox.com', 'Comments': None, 'PhonePrimaryContact': None, 'PhoneSecondaryContact': None, 'IsPrivateIndividual': False, 'AddressInfo': None, 'BookingURL': None, 'ContactEmail': None, 'FaultReportEmail': None, 'IsRestrictedEdit': False, 'ID': 3339, 'Title': 'Chargefox'}, 'UsageType': {'IsPayAtLocation': False, 'IsMembershipRequired': True, 'IsAccessKeyRequired': True, 'ID': 4, 'Title': 'Public - Membership Required'}, 'StatusType': {'IsOperational': True, 'IsUserSelectable': True, 'ID': 50, 'Title': 'Operational'}, 'SubmissionStatus

Once I had the data, I loaded it into a pandas DataFrame for easier manipulation and analysis. I printed the column names to understand the structure of the data and displayed the first few rows of the DataFrame to inspect the contents.

In [4]:
# Assuming `data` is your JSON data
df = pd.DataFrame(data)

# Print the column names
print(df.columns)

# Display the first few rows of the DataFrame
print(df.head())


Index(['DataProvider', 'OperatorInfo', 'UsageType', 'StatusType',
       'SubmissionStatus', 'UserComments', 'PercentageSimilarity',
       'MediaItems', 'IsRecentlyVerified', 'DateLastVerified', 'ID', 'UUID',
       'ParentChargePointID', 'DataProviderID', 'DataProvidersReference',
       'OperatorID', 'OperatorsReference', 'UsageTypeID', 'UsageCost',
       'AddressInfo', 'Connections', 'NumberOfPoints', 'GeneralComments',
       'DatePlanned', 'DateLastConfirmed', 'StatusTypeID',
       'DateLastStatusUpdate', 'MetadataValues', 'DataQualityLevel',
       'DateCreated', 'SubmissionStatusTypeID'],
      dtype='object')
                                        DataProvider  \
0  {'WebsiteURL': 'http://openchargemap.org', 'Co...   
1  {'WebsiteURL': 'http://openchargemap.org', 'Co...   
2  {'WebsiteURL': 'http://openchargemap.org', 'Co...   
3  {'WebsiteURL': 'http://openchargemap.org', 'Co...   
4  {'WebsiteURL': 'http://openchargemap.org', 'Co...   

                                   

In this cell, I explored the **AddressInfo** column, which contains information about the charging station's address, such as latitude, longitude, and the specific address. I displayed the first few rows to see the structure of this nested information.

In [5]:
# Display the first few rows of the 'AddressInfo' column
print(df['AddressInfo'].head())


0    {'ID': 304105, 'Title': 'NRMA Midland ', 'Addr...
1    {'ID': 303833, 'Title': 'K Park Playhouse Lane...
2    {'ID': 302972, 'Title': 'BP South Bound', 'Add...
3    {'ID': 302784, 'Title': 'Miller Grove', 'Addre...
4    {'ID': 302598, 'Title': 'Green Head Supercharg...
Name: AddressInfo, dtype: object


The output shows the first few rows of the **AddressInfo** column, which contains dictionaries with fields like **ID, Title, AddressLine1,** etc.

I created a function to extract specific fields (latitude, longitude, address, city, state, country, and postal code) from the nested **AddressInfo** column. After applying this function to each row, I converted the extracted data into a new DataFrame and combined it with the original DataFrame. This allows me to access the extracted address information easily.

In [6]:
import pandas as pd
import requests

# Fetch data from API
url = "https://api.openchargemap.io/v3/poi/?output=json&countrycode=AU"
headers = {
    "X-API-Key": "2f5df05b-4b9d-46a3-b38e-c138e919abea"  # Replace with your actual API key
}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
else:
    raise Exception(f"Failed to retrieve data. Status code: {response.status_code}")

# Create DataFrame
df = pd.DataFrame(data)

# Define a function to extract specific fields from AddressInfo
def extract_address_info(address_info):
    if isinstance(address_info, dict):
        return {
            'latitude': address_info.get('Latitude'),
            'longitude': address_info.get('Longitude'),
            'address_line1': address_info.get('AddressLine1', 'N/A'),
            'city': address_info.get('Town', 'N/A'),
            'state': address_info.get('StateOrProvince', 'N/A'),
            'country': address_info.get('Country', 'N/A'),
            'postal_code': address_info.get('Postcode', 'N/A')
        }
    return {}

# Apply the extraction function and convert to DataFrame
address_info_df = df['AddressInfo'].apply(extract_address_info).apply(pd.Series)

# Combine with the original DataFrame
df_combined = pd.concat([df, address_info_df], axis=1)

# Display the combined DataFrame with extracted fields
print(df_combined.head())


                                        DataProvider  \
0  {'WebsiteURL': 'http://openchargemap.org', 'Co...   
1  {'WebsiteURL': 'http://openchargemap.org', 'Co...   
2  {'WebsiteURL': 'http://openchargemap.org', 'Co...   
3  {'WebsiteURL': 'http://openchargemap.org', 'Co...   
4  {'WebsiteURL': 'http://openchargemap.org', 'Co...   

                                        OperatorInfo  \
0  {'WebsiteURL': 'https://chargefox.com', 'Comme...   
1  {'WebsiteURL': 'https://chargefox.com', 'Comme...   
2  {'WebsiteURL': 'https://www.bp.com/en_au/austr...   
3  {'WebsiteURL': 'https://exploren.com.au/', 'Co...   
4  {'WebsiteURL': 'http://www.teslamotors.com', '...   

                                           UsageType  \
0  {'IsPayAtLocation': False, 'IsMembershipRequir...   
1  {'IsPayAtLocation': False, 'IsMembershipRequir...   
2  {'IsPayAtLocation': False, 'IsMembershipRequir...   
3  {'IsPayAtLocation': False, 'IsMembershipRequir...   
4  {'IsPayAtLocation': False, 'IsMembershipReq

Here, I used the **folium** library to create an interactive map. I centered the map around the average location of the charging stations and added markers for each station, with popups displaying their address information. The map was saved as an HTML file for visualization in a browser.

In [7]:
import folium

# Create a map centered around the average location of charging stations
map_center = [df_combined['latitude'].mean(), df_combined['longitude'].mean()]
map_ = folium.Map(location=map_center, zoom_start=6)

# Add charging station markers to the map
for _, row in df_combined.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(
            f"{row['address_line1']}, {row['city']}, {row['state']}, {row['country']}, {row['postal_code']}",
            max_width=300
        ),
        icon=folium.Icon(color='blue', icon='bolt')
    ).add_to(map_)

# Save the map as an HTML file
map_.save("charging_stations_map.html")


I used the TomTom Traffic API to get traffic data for a specific location (New York City in this case) by providing the coordinates and API key. The response included data like the current and free-flow speeds on the road. I printed the current speed and free-flow speed, along with the full response to see what other information is available.

In [20]:
url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
params = {
    "point": "40.712776,-74.005974",  # Example coordinates for New York City
    "unit": "KMPH",  # Units
    "key": "Eap7ro6nQSWxx9cPDrSDha8NgfjaoQK9"
}
response = requests.get(url, params=params)
traffic_data = response.json()

# Access traffic data
if "flowSegmentData" in traffic_data:
    print(traffic_data["flowSegmentData"]["currentSpeed"])
    print(traffic_data["flowSegmentData"]["freeFlowSpeed"])
    print(traffic_data)


40
40
{'flowSegmentData': {'frc': 'FRC4', 'currentSpeed': 40, 'freeFlowSpeed': 40, 'currentTravelTime': 22, 'freeFlowTravelTime': 22, 'confidence': 1, 'roadClosure': False, 'coordinates': {'coordinate': [{'latitude': 40.71192297683463, 'longitude': -74.00607219291992}, {'latitude': 40.712024886432246, 'longitude': -74.00567656709015}, {'latitude': 40.712042358448585, 'longitude': -74.00561353517827}, {'latitude': 40.712062435378066, 'longitude': -74.00552770448978}, {'latitude': 40.71207584117256, 'longitude': -74.00545930815987}, {'latitude': 40.712083910061786, 'longitude': -74.0053775007849}, {'latitude': 40.712086578513116, 'longitude': -74.00531715108204}, {'latitude': 40.71208124161032, 'longitude': -74.00524339033413}, {'latitude': 40.71206917004296, 'longitude': -74.0051682884817}, {'latitude': 40.71205443002083, 'longitude': -74.00510659767434}, {'latitude': 40.71204369267512, 'longitude': -74.00507038785264}, {'latitude': 40.71201954952451, 'longitude': -74.00501003814978}, {

The output showed that the current speed and free-flow speed are both 40 km/h for the selected location. The full response also includes a detailed breakdown of road coordinates and confidence in the traffic data.

Then, I created a function to fetch traffic data for any location by specifying the latitude, longitude, and API key. This function is reusable for multiple locations, and it returns the traffic data in JSON format.

In [21]:
import requests

# Function to get traffic data for a given location
def get_traffic_data(lat, lon, api_key):
    url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
    params = {
        "point": f"{lat},{lon}",
        "unit": "KMPH",
        "key": api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None


In this cell, I integrated traffic data into the charging station map. For each charging station, I fetched traffic data using the previously defined function. The popup for each marker now includes traffic information, such as current and free-flow speeds, along with the address. I saved the updated map as an HTML file.

In [22]:
import folium

# Your TomTom API key
api_key = "Eap7ro6nQSWxx9cPDrSDha8NgfjaoQK9"

# Create a map centered around the average location of charging stations
map_center = [df_combined['latitude'].mean(), df_combined['longitude'].mean()]
map_ = folium.Map(location=map_center, zoom_start=6)

# Add charging station markers to the map
for _, row in df_combined.iterrows():
    # Get traffic data for the charging station location
    traffic_data = get_traffic_data(row['latitude'], row['longitude'], api_key)
    
    # Extract relevant traffic information
    if traffic_data and "flowSegmentData" in traffic_data:
        current_speed = traffic_data["flowSegmentData"]["currentSpeed"]
        free_flow_speed = traffic_data["flowSegmentData"]["freeFlowSpeed"]
        traffic_info = f"Current Speed: {current_speed} km/h\nFree Flow Speed: {free_flow_speed} km/h"
    else:
        traffic_info = "Traffic data not available"
    
    # Create a popup with charging station and traffic information
    popup_text = (f"{row['address_line1']}, {row['city']}, {row['state']}, {row['country']}, {row['postal_code']}<br>"
                  f"<b>Traffic Info:</b><br>{traffic_info}")
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color='blue', icon='bolt')
    ).add_to(map_)

# Save the map as an HTML file
map_.save("charging_stations_map_with_traffic.html")


I improved the map by assigning different marker colors based on the traffic conditions. I defined a function to calculate the traffic condition (smooth, moderate, or heavy) by comparing the current speed to the free-flow speed. Green markers indicate smooth traffic, light green for moderate traffic, and red for heavy traffic. If traffic data is unavailable, the marker is gray. The final map was saved as an HTML file.

In [28]:
import folium
import requests

# Function to fetch traffic data
def get_traffic_data(lat, lon, api_key):
    url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
    params = {
        "point": f"{lat},{lon}",
        "unit": "KMPH",  # Units
        "key": api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Function to determine the traffic condition and marker color
def get_marker_color(current_speed, free_flow_speed):
    if free_flow_speed == 0:
        return "gray"  # No data or road closure

    speed_ratio = current_speed / free_flow_speed
    if speed_ratio >= 0.8:
        return "green"  # Smooth traffic
    elif speed_ratio >= 0.5:
        return "lightgreen"  # Moderate traffic
    else:
        return "red"  # Heavy traffic

# Your TomTom API key
api_key = "Eap7ro6nQSWxx9cPDrSDha8NgfjaoQK9"

# Create a map centered around the average location of charging stations
map_center = [df_combined['latitude'].mean(), df_combined['longitude'].mean()]
map_ = folium.Map(location=map_center, zoom_start=6)

# Add charging station markers to the map
for _, row in df_combined.iterrows():
    # Get traffic data for the charging station location
    traffic_data = get_traffic_data(row['latitude'], row['longitude'], api_key)
    
    # Extract relevant traffic information and determine marker color
    if traffic_data and "flowSegmentData" in traffic_data:
        current_speed = traffic_data["flowSegmentData"]["currentSpeed"]
        free_flow_speed = traffic_data["flowSegmentData"]["freeFlowSpeed"]
        traffic_info = f"Current Speed: {current_speed} km/h<br>Free Flow Speed: {free_flow_speed} km/h"
        marker_color = get_marker_color(current_speed, free_flow_speed)
    else:
        traffic_info = "Traffic data not available"
        marker_color = "gray"  # Default color if traffic data is unavailable

    # Create a popup with charging station and traffic information
    popup_text = (f"{row['address_line1']}, {row['city']}, {row['state']}, {row['country']}, {row['postal_code']}<br>"
                  f"<b>Traffic Info:</b><br>{traffic_info}")
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color=marker_color, icon='bolt')
    ).add_to(map_)

# Save the map as an HTML file
map_.save("charging_stations_map_with_traffic_and_colors.html")
